# Interactive visualization with `bokeh`

Let's use the [bokeh](https://bokeh.org/) visualization library to explore the spectra.

In [ ]:
import numpy as np
import pandas as pd
import bokeh

Read in the model spectra same as before:

In [ ]:
teff_points = [500, 525, 550, 575, 600, 650, 700, 750, 800,
            850, 900, 950,  1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700,
            1800,1900, 2000, 2100, 2200, 2300, 2400]
logg_points = np.arange(4.0, 5.51, 0.25)

In [ ]:
logg_par_dict = {4.0:"100",4.25:"178",4.5:"316",4.75:"562",
                           5.0:"1000",5.25:"1780",5.5:"3160"}

In [ ]:
teff = 1000
logg = 5

In [ ]:
base_name = "sp_t{0:0>.0f}g{1:}nc_m0.0".format(teff, logg_par_dict[logg])

fn = '../models/spectra/'+base_name+'.gz'

In [ ]:
def load_and_prep_spectrum(fn, downsample=5):
    df_native = pd.read_csv(fn, 
                        skiprows=[0, 1], 
                        delim_whitespace=True, 
                        compression='gzip',
                        names=['wavelength', 'flux']
                       ).sort_values('wavelength').reset_index(drop=True)

    nir_mask = (df_native.wavelength > 1.2) & (df_native.wavelength < 1.35)

    ## decimate the data:
    df_nir = df_native[nir_mask].rolling(5, win_type='gaussian').mean(std=3).iloc[::downsample, :]
    return df_nir.dropna().reset_index(drop=True)

In [ ]:
df_nir = load_and_prep_spectrum(fn)

In [ ]:
df_nir.shape

We will build heavily off of the [interact](https://github.com/lightkurve/lightkurve/blob/main/src/lightkurve/interact.py) method from the [lightkurve](https://docs.lightkurve.org/) framework.

In [ ]:
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import (
    Slider,
    Span,
    Range1d,
    Dropdown
)
from bokeh.layouts import layout, Spacer
from bokeh.models.widgets import Div

from scipy.ndimage import gaussian_filter1d
from collections import OrderedDict

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [ ]:
teff_dict = OrderedDict()

In [ ]:
for i, teff in enumerate(teff_points):
    teff_dict[i]=str(teff)

In [ ]:
fig = figure(
        title="Sonora Bobcat in Bokeh",
        plot_height=340,
        plot_width=600,
        tools="pan,wheel_zoom,box_zoom,tap,reset",
        toolbar_location="below",
        border_fill_color="whitesmoke",
    )

In [ ]:
teff_slider = Slider(
            start=min(teff_dict.keys()),
            end=max(teff_dict.keys()),
            value=10,
            step=1,
            title="Teff",
            width=490
        )

In [ ]:
def create_interact_ui(doc):
    
    # Make the spectrum source
    spec_source = ColumnDataSource(
        data=dict(
            wavelength=df_nir.wavelength.values,
            flux=gaussian_filter1d(df_nir.flux.values, 0.1),
            native_flux = df_nir.flux.values,
            native_wavelength = df_nir.wavelength.values
        )
    )
    
    fig = figure(
        title="Sonora Bobcat in Bokeh",
        plot_height=340,
        plot_width=600,
        tools="pan,wheel_zoom,box_zoom,tap,reset",
        toolbar_location="below",
        border_fill_color="whitesmoke",
    )
    fig.title.offset = -10
    fig.yaxis.axis_label = "Flux "
    fig.xaxis.axis_label = "Wavelength (micron)"
    ymax = df_nir.flux.max()*1.2
    fig.y_range = Range1d(start=0, end=ymax)
    xmin, xmax = df_nir.wavelength.min()*0.995, df_nir.wavelength.max()*1.005
    fig.x_range = Range1d(start=xmin, end=xmax)

    fig.step(
            "wavelength",
            "flux",
            line_width=1,
            color="gray",
            source=spec_source,
            nonselection_line_color="gray",
            nonselection_line_alpha=1.0,
        )
    
    # Slider to decimate the data
    smoothing_slider = Slider(
            start=0.1,
            end=40,
            value=0.1,
            step=0.1,
            title="Spectral resolution kernel",
            width=490
        )
    
    vz_slider = Slider(
            start=-0.009,
            end=0.009,
            value=0.00,
            step=0.0005,
            title="Radial Velocity",
            width=490,
        format='0.000f'
        )
    

    teff_slider = Slider(
            start=min(teff_points),
            end=max(teff_points),
            value=1000,
            step=25,
            title="Teff",
            width=490
        )
    teff_message = Div(text=str(1000), width=100, height=10)
    
    logg_slider = Slider(
            start=min(logg_points),
            end=max(logg_points),
            value=5.0,
            step=0.25,
            title="logg",
            width=490
        )
    
    def update_upon_smooth(attr, old, new):
        """Callback to take action when smoothing slider changes"""
        #spec_source.data["wavelength"] = df_nir.wavelength.values[::new]
        spec_source.data["flux"] = gaussian_filter1d(spec_source.data["native_flux"], new)
        
    def update_upon_vz(attr, old, new):
        """Callback to take action when vz slider changes"""
        spec_source.data["wavelength"] = spec_source.data["native_wavelength"] - new
        #spec_source.data["flux"] = gaussian_filter1d(df_nir.flux.values, new)
        
    def update_upon_teff_selection(attr, old, new):
        """Callback to take action when teff slider changes"""
        teff = find_nearest(teff_points, new)
        if teff != old:
            teff_message.text = str(new)
            base_name = "sp_t{0:0>.0f}g{1:}nc_m0.0".format(np.float(teff), logg_par_dict[logg])

            fn = '../models/spectra/'+base_name+'.gz'
            df_nir = load_and_prep_spectrum(fn, downsample=5)
            ymax = df_nir.flux.max()*1.2
            fig.y_range.end =ymax
            spec_source.data["native_wavelength"] = df_nir.wavelength.values
            spec_source.data["wavelength"] = df_nir.wavelength.values - vz_slider.value
            spec_source.data["flux"] = gaussian_filter1d(df_nir.flux.values, smoothing_slider.value)
            spec_source.data["native_flux"] = df_nir.flux.values
        else:
            pass
        
    def update_upon_logg_selection(attr, old, new):
        """Callback to take action when logg slider changes"""
        teff = find_nearest(teff_points, teff_slider.value)
        base_name = "sp_t{0:0>.0f}g{1:}nc_m0.0".format(np.float(teff), logg_par_dict[new])

        fn = '../models/spectra/'+base_name+'.gz'
        df_nir = load_and_prep_spectrum(fn, downsample=5)
        ymax = df_nir.flux.max()*1.2
        fig.y_range.end = ymax
        spec_source.data["native_wavelength"] = df_nir.wavelength.values
        spec_source.data["wavelength"] = df_nir.wavelength.values - vz_slider.value
        spec_source.data["flux"] = gaussian_filter1d(df_nir.flux.values, smoothing_slider.value)
        spec_source.data["native_flux"] = df_nir.flux.values

            
        
    smoothing_slider.on_change("value", update_upon_smooth)
    vz_slider.on_change("value", update_upon_vz)
    teff_slider.on_change("value", update_upon_teff_selection)
    logg_slider.on_change("value", update_upon_logg_selection)
    
    sp1= Spacer(width=15)
    
    widgets_and_figures = layout(
            [fig],
            [teff_slider, sp1, teff_message],
            [logg_slider],
            [smoothing_slider],
            [vz_slider]
        )
    doc.add_root(widgets_and_figures)

In [ ]:
output_notebook(verbose=False, hide_banner=True)
show(create_interact_ui)

Woohoo, it works!